## Part 1: Preprocessing

In [136]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [181]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [182]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [183]:
# Create a list of at least 10 column names to use as X data
X_cols = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[X_cols]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [184]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [185]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train['OverTime'] = X_train['OverTime'].map({'No': 0, 'Yes': 1})
X_test['OverTime'] = X_test['OverTime'].map({'No': 0, 'Yes': 1})

X_train['OverTime'].value_counts()

OverTime
0    785
1    317
Name: count, dtype: int64

In [186]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [187]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_department = department_encoder.transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])

In [188]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [190]:
# Find the number of columns in the X training data
input_nodes = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(input_nodes,), name='input_layer')


# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [191]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name='department_output')(dept_hidden)

In [192]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [193]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'},
    metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 10)]         0           []                               
                                                                                                  
 shared1 (Dense)                (None, 64)           704         ['input_layer[0][0]']            
                                                                                                  
 shared2 (Dense)                (None, 128)          8320        ['shared1[0][0]']                
                                                                                                  
 department_hidden (Dense)      (None, 32)           4128        ['shared2[0][0]']                
                                                                                            

In [194]:
# Train the model
train_history = model.fit(
    X_train_scaled, {'department_output': y_train_department, 'attrition_output': y_train_attrition},
    epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 5ms/step - loss: 1.3646 - department_output_loss: 0.8740 - attrition_output_loss: 0.4905 - department_output_accuracy: 0.6180 - attrition_output_accuracy: 0.8285
Epoch 2/100
35/35 [==============================] - 0s 4ms/step - loss: 1.2041 - department_output_loss: 0.7767 - attrition_output_loss: 0.4273 - department_output_accuracy: 0.6434 - attrition_output_accuracy: 0.8285
Epoch 3/100
35/35 [==============================] - 0s 4ms/step - loss: 1.1692 - department_output_loss: 0.7648 - attrition_output_loss: 0.4044 - department_output_accuracy: 0.6497 - attrition_output_accuracy: 0.8312
Epoch 4/100
35/35 [==============================] - 0s 4ms/step - loss: 1.1501 - department_output_loss: 0.7585 - attrition_output_loss: 0.3916 - department_output_accuracy: 0.6452 - attrition_output_accuracy: 0.8457
Epoch 5/100
35/35 [==============================] - 0s 4ms/step - loss: 1.1354 - department_output_loss: 0.7503 - attrition_out

In [195]:
# Evaluate the model with the testing data
loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = \
    model.evaluate(X_test_scaled, {'department_output': y_test_department,
                    'attrition_output': y_test_attrition})

12/12 [==============================] - 0s 3ms/step - loss: 4.1522 - department_output_loss: 3.0692 - attrition_output_loss: 1.0830 - department_output_accuracy: 0.5598 - attrition_output_accuracy: 0.8152


In [196]:
# Print the accuracy for both department and attrition
print(f'Department predictions accuracy: {department_accuracy}')
print(f'Attrition predictions accuracy: {attrition_accuracy}')

Department predictions accuracy: 0.5597826242446899
Attrition predictions accuracy: 0.8152173757553101


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No because the business angle for predicting attrition is that a company might want to identify employees at risk of leaving, and potentially intervene in some way to retain those employees. With that being the overarching goal, accuracy does not tell us what's actually important. As I demonstrate in the next cell, this model had 82% overall accuracy in predicting attrition, but it only had 40% recall. Recall is the ratio of true positive predictions to all positives in the ground truth. That means that out of the employees who left, the model was only able to correctly identify 40% of them. That might be an acceptable amount, but more importantly it gives a better idea of how useful the model would be for our goals.

    Given that the full dataset is 84% employees who didn't leave, hypothetically a *very* bad model could predict that all employees stay, and still have 84% accuracy. We wouldn't understand how bad the model actually is until we saw that it had 0% recall.
2. I chose softmax for department because there are 3 mutually exclusive departments to predict, and softmax is suited for multi-class predictions. For attrition, I chose sigmoid because attrition is binary (left the company or didn't), and sigmoid is better suited for binary predictions, as it outputs a single number between 0 and 1.
3.  - More data
    - Balancing the data across the target variables prior to fitting (especially if there is more data)
    - Increasing the quantity of nodes and layers
    - Implementing checkpoints `tf.keras.callbacks.ModelCheckpoint()` and configuring it to save the weights of the best validation recall score during fitting. 

In [197]:
from sklearn.metrics import classification_report

department_preds, attrition_preds = model.predict(X_test_scaled)

print(classification_report(y_test_attrition, attrition_preds.round()))

              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89       320
         1.0       0.33      0.40      0.36        48

    accuracy                           0.82       368
   macro avg       0.62      0.64      0.63       368
weighted avg       0.83      0.82      0.82       368

